"""
An example to demonstrate online linear system identification

We demonstrate the use of OnlineLinearModel class with a simple linear system.
Take a 2D time-varying system dx/dt=A(t)x(t)+B(t)u(t), where A(t) and B(t)
are slowly varying with time. In particular, we take A(t)=(1+eps\*t)\*A,
B(t)=(1+eps\*t)\*B, and eps = 0.1 is small. It is discretize with
time step dt = 0.1. Denote the discrete system as x(k+1)=A(k)x(k)+
B(k)u(k).

At time step t+1, we need to include new snapshot pair x(t), u(t), x(t+1).
We would like to update the adaptive model in real-time

Authors: 
Hao Zhang

References:
Zhang, Hao, Clarence W. Rowley, Eric A. Deem, and Louis N. Cattafesta.
"Online dynamic mode decomposition for time-varying systems."
SIAM Journal on Applied Dynamical Systems 18, no. 3 (2019): 1586-1609.

Created:
June 2017.
"""

In [1]:
!file $(which python)

/Users/haozhang/python-venvs/oml-reorg/bin/python: Mach-O 64-bit executable x86_64


In [2]:
!python --version

Python 3.8.5


In [3]:
!python -m pip install -Uqqq pip setuptools wheel

In [4]:
!python -m pip install -r requirements.txt

  Using cached slycot-0.5.0.0.tar.gz (3.0 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for slycot: filename=slycot-0.5.0-cp38-cp38-macosx_10_16_x86_64.whl size=1939812 sha256=ca8836f22bcc30130238c8047de452c9deb906e3b7ac9a054127dfe36e7af79d
  Stored in directory: /Users/haozhang/Library/Caches/pip/wheels/b4/39/2a/32ae2ed2c02ec0e386de63474126746edf2c83b632b9d2ad44
Successfully built slycot


In [5]:
!pip list

Package            Version   Editable project location
------------------ --------- -------------------------------------
ai4sci.oml           0.2.0     /Users/haozhang/Local/open-source/oml
attrs              22.1.0
bleach             5.0.1
build              0.8.0
certifi            2022.9.24
charset-normalizer 2.1.1
commonmark         0.9.1
contourpy          1.0.5
control            0.9.2
cycler             0.11.0
docutils           0.19
fonttools          4.37.4
idna               3.4
importlib-metadata 5.0.0
iniconfig          1.1.1
jaraco.classes     3.2.3
keyring            23.9.3
kiwisolver         1.4.4
matplotlib         3.6.1
more-itertools     9.0.0
numpy              1.23.4
oml                0.2.0
packaging          21.3
pep517             0.13.0
Pillow             9.2.0
pip                22.3
pkginfo            1.8.3
pluggy             1.0.0
py                 1.11.0
Pygments           2.13.0
pyparsing          3.0.9
pytest             7.1.3
python-dateutil    2.8.2
re

In [6]:
"""
An example to demonstrate online linear system identification

We demonstrate the use of OnlineLinearModel class with a simple linear system.
Take a 2D time-varying system dx/dt=A(t)x(t)+B(t)u(t), where A(t) and B(t)
are slowly varying with time. In particular, we take A(t)=(1+eps*t)*A,
B(t)=(1+eps*t)*B, and eps = 0.1 is small. It is discretize with
time step dt = 0.02. Denote the discrete system as x(k+1)=A(k)x(k)+
B(k)u(k).

At time step k+1, we need to include new snapshot pair x(k), u(k), x(k+1).
We would like to update the adaptive model in real-time

Authors: 
Hao Zhang

References:
Zhang, Hao, Clarence W. Rowley, Eric A. Deem, and Louis N. Cattafesta.
"Online dynamic mode decomposition for time-varying systems."
SIAM Journal on Applied Dynamical Systems 18, no. 3 (2019): 1586-1609.

Created:
June 2017.
"""

import random
import matplotlib.pyplot as plt
import numpy as np
from oml import OnlineLinearModel
from control import lqr, StateSpace, ctrb

random.seed(20210220)
np.random.seed(20210220)


# define dynamics, negative damping -> unstable
# https://en.wikipedia.org/wiki/Harmonic_oscillator#Damped_harmonic_oscillator
n = 2 # state dimension
k = 1 # control dimension
A = np.array([[0, 1], [-1, 0.2]])
B = np.array([[0], [1]])
C = np.eye(n)
D = np.zeros((n, k))

# check poles
sys = StateSpace(A, B, np.eye(n), np.zeros((n, k)))
print(f"open loop poles: {sys.pole()}")

# check controllability
Ctrb = ctrb(A, B)
assert np.linalg.matrix_rank(Ctrb) == n

# slowly time-varying dynamics
def dyn(t, x, u, epsilon=1e-3):
    At = (1 + epsilon * t) * A
    Bt = (1 + epsilon * t) * B
    dxdt = At.dot(x) + Bt.dot(u)
    return dxdt

# set up simulation parameter
dt = 0.02
tmax, tc = 20, 0.5
T, kc = int(tmax / dt), int(tc / dt)
tspan = np.linspace(0, tmax, T + 1)

# online linear system identification setup
alpha = 0.01 ** (1.0 / kc) # 99% decay after kc samples
olm = OnlineLinearModel(n, k, None, alpha)

# store data mtrices
x = np.zeros([n, T])
u = np.zeros([k, T])

# initial condition, state and control
x0 = np.array([1, 0])
u0 = 0

# uncontrolled system
x[:, 0] = x0
u[:, 0] = u0
for t in range(1, T):
    # forward the system for one step
    x[:, t] = x[:, t - 1] + dt * dyn(t * dt, x[:, t - 1], u[:, t - 1])
    u[:, t] = 0

def plot_state_control(x, u):
    plt.rcParams['figure.dpi'] = 100
    fig, axs = plt.subplots(2, figsize=(10, 6))
    fig.suptitle('State and control')
    axs[0].plot(tspan[1:], x[0, :], "b-", linewidth=1.0, label="State $x_1(t)$")
    axs[0].plot(tspan[1:], x[1, :], "g-", linewidth=1.0, label="State $x_2(t)$")
    axs[0].legend(loc="best", fontsize=12, shadow=True)
    axs[0].grid()

    axs[1].plot(tspan[1:], u.reshape(-1), "r-", linewidth=1.0, label="Control $u(t)$")
    axs[1].legend(loc="best", fontsize=12, shadow=True)
    axs[1].grid()
    axs[1].set_xlabel("Time", fontsize=12)

plot_state_control(x, u)

open loop poles: [0.1+0.99498744j 0.1-0.99498744j]


In [7]:
# control system simulation
# initial condition, state and control
x[:, 0] = np.array([1, 0])
u[:, 0] = 0
for t in range(1, T):
    # forward the system for one step
    x[:, t] = x[:, t - 1] + dt * dyn(t * dt, x[:, t - 1], u[:, t - 1])
    # use new measurement to update online system identification
    olm.update(x[:, t - 1], u[:, t - 1], x[:, t])
    # apply control if we have collected enough data
    if t > 4 * max(n, n + k):
        # convert to continuous representation
        Ac = (olm.A - np.eye(n)) / dt
        Bc = olm.B / dt
        # get LQR gain, output = state
        sys = StateSpace(Ac, Bc, C, D, dt=0)
        K, S, E = lqr(sys, np.eye(n), np.eye(k))
        # apply control
        u[:, t] = -1.0 * K.dot(x[:, t])
        # clip control
        u[:, t] = np.clip(u[:, t], -10, 10)
        # smooth control
        w = 0.5
        u[:, t] = (1 - w) * u[:, t - 1] + w * u[:, t]
        # show progress
        if t % 100 == 0:
          print(f"x(t)={x[:, t]}")
          print(f"u(t)={u[:, t]}")
          print(f"Ac={Ac}")
          print(f"Bc={Bc}")
          print(f"K={K}")
    # random small perturbation if not enough data yet
    else:
        u[:, t] = 1e-1 * np.random.randn()

plot_state_control(x, u)

ControlSlycot: can't find slycot module 'sb02md' or 'sb02nt'